In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os


In [ ]:
# Assuming the images are in a directory
image_directory = 'path/to/your/images'

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

images = load_images_from_folder(image_directory)


In [ ]:
# Example: Using MNIST dataset from Keras
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert grayscale images to RGB and resize to fit the MobileNet-SSD input size
x_train = [cv2.cvtColor(cv2.resize(np.stack((img,)*3, axis=-1), (300, 300)), cv2.COLOR_GRAY2RGB) for img in x_train]
x_test = [cv2.cvtColor(cv2.resize(np.stack((img,)*3, axis=-1), (300, 300)), cv2.COLOR_GRAY2RGB) for img in x_test]


In [ ]:
# Example: Importing data from Kaggle
# !kaggle datasets download -d dataset/dataset-name

# Unzip the downloaded dataset
# !unzip dataset-name.zip -d dataset-directory

# Load images from the dataset directory
kaggle_images = load_images_from_folder('dataset-directory')


In [ ]:
# Download the MobileNet-SSD model files
!wget -q --show-progress https://github.com/chuanqi305/MobileNet-SSD/blob/master/deploy.prototxt
!wget -q --show-progress https://github.com/chuanqi305/MobileNet-SSD/blob/master/mobilenet_iter_73000.caffemodel


In [ ]:
def detect_objects(image, net, confidence_threshold=0.2):
    # Get the image dimensions
    (h, w) = image.shape[:2]
    
    # Preprocess the image to fit the model input size
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    
    # Set the input to the pre-trained deep learning network and obtain the output predicted probabilities
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # Extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections by ensuring the `confidence` is greater than the minimum confidence
        if confidence > confidence_threshold:
            # Extract the index of the class label from the `detections`
            idx = int(detections[0, 0, i, 1])
            
            # Compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # Draw the prediction on the image
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image

# Detect objects in all images
detected_images = [detect_objects(img, net) for img in images]

# Display one of the detected images
plt.imshow(cv2.cvtColor(detected_images[0], cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
# Assuming you have ground truth labels and a way to calculate metrics
# This section is just a placeholder, as evaluation metrics will depend on your dataset and requirements
def evaluate_model(images, ground_truth_labels):
    # Placeholder for evaluation logic
    pass

# Example visualization of detected images
for img in detected_images:
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
